# To generate stellar parameters and abundances for whole Apogee DR14

### Please notice this notebook requires intensive computational resources to run for approx. few hours

First we need to do continuum normalization on the whole Apogee DR14 combined spectra and save the file

In [ ]:
import numpy as np
import time
import h5py
from astropy.io import fits

from astroNN.apogee import allstar, visit_spectra
from astroNN.datasets import xmatch
from astroNN.gaia import mag_to_fakemag
from astroNN.apogee.chips import apogee_continuum, gap_delete
from astroNN.nn.losses import mean_absolute_percentage_error

target_bit = [0, 1, 2, 3, 4, 5, 6, 7, 12]
def apstar_normalization(spectra, spectra_err, _spec_mask):
    return apogee_continuum(spectra=spectra, spectra_err=spectra_err, cont_mask=None, deg=2, dr=14, bitmask=_spec_mask)

allstar_data = fits.getdata(allstar(dr=14))

start_time = time.time()

total_num = allstar_data['RA'].shape[0]
spec = np.zeros((total_num, 7514), dtype=np.float32)

for counter in range(0, total_num):
    if allstar_data['LOCATION_ID'][counter] == 1:
        continue
    ap_path = visit_spectra(dr=14, apogee=allstar_data['APOGEE_ID'][counter], 
                            location=allstar_data['LOCATION_ID'][counter], verbose=0)
    if ap_path is False:
        pass
    else:
        apstar_file = fits.open(ap_path)
        nvisits = apstar_file[0].header['NVISITS']
        # we only want combined spectra
        if nvisits == 1:
            _spec = apstar_file[1].data
            _spec_err = apstar_file[2].data
            _spec_mask = apstar_file[3].data
        else:
            _spec = apstar_file[1].data[1]
            _spec_err = apstar_file[2].data[1]
            _spec_mask = apstar_file[3].data[1]
        
        if not np.all(_spec==0.):
            _spec, _spec_err = apstar_normalization(_spec, _spec_err, _spec_mask)
            spec[counter] = _spec

    if counter % 100 == 0:
        print(f'Completed {counter} of {total_num}, {(time.time() - start_time):.{2}f}s elapsed')
        
# save a fits
hdu = fits.PrimaryHDU(spec)
hdu.writeto('contspec_dr14.fits')

Then we do inference on all combined spectra loaded from the compiled file and generate a catalog

In [ ]:
import numpy as np
import time
import h5py
from astropy.io import fits

from astroNN.apogee import allstar, visit_spectra
from astroNN.datasets import xmatch
from astroNN.gaia import mag_to_fakemag
from astroNN.apogee.chips import apogee_continuum, gap_delete
from astroNN.nn.losses import mean_absolute_percentage_error
from astroNN.models import load_folder

net = load_folder("your_astroNN_model_here")

file = fits.open("contspec_dr14.fits")
allstar_file = fits.open(allstar(dr=14))
all_spec = file[0].data
        
pred, pred_error = net.test(all_spec)

# some spectra are all zeros, set prediction for those spectra to -9999.
pred[np.all(all_spec == 0., axis=1)] = -9999.
pred_error['total'][np.all(all_spec == 0., axis=1)] = -9999.

# save a fits
c1 = fits.Column(name='APOGEE_ID', array=allstar_data['APOGEE_ID'], format="18A")
c2 = fits.Column(name='LOCATION_ID', array=allstar_data['LOCATION_ID'], format="J")
c3 = fits.Column(name='RA', array=allstar_data['RA'], format='D')
c4 = fits.Column(name='DEC', array=allstar_data['DEC'], format='D')
c5 = fits.Column(name='astroNN', array=pred, format='22E')
c6 = fits.Column(name='astroNN_error', array=pred_error['total'], format='22E')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6])
t.writeto('astroNN_apogee_dr14_catalog.fits')